In [1]:
import numpy as np


In [2]:
from product import Product
from json import load

with open('agora_hack_products.json', encoding='utf-8') as file:
    all_products = load(file)

all_products = [Product(**p) for p in all_products]

In [3]:
references = [p for p in all_products if p.is_reference]
references_id_set = set([ref.product_id for ref in references])
products = [p for p in all_products if p.product_id not in references_id_set]

In [4]:
from sklearn.model_selection import train_test_split

# удалим часть эталонов, что бы в датасете были 'ничейные' товары
references, nulled_references = train_test_split(
    references, test_size=0.5 , random_state=42
)
references += nulled_references
nulled_references = []

nulled_references_set = set([r.product_id for r in nulled_references])
for p in products:
    if p.reference_id in nulled_references_set:
        p.reference_id = None

products_train, products_test = train_test_split(
    products, test_size=0.5, random_state=42
)
all_products = products_train + references
products_test = products_test + nulled_references

In [5]:
def accuracy(predicted, target):
    return list(map(lambda v: v[0] == v[1], zip(predicted, target))).count(True) / len(predicted)

In [6]:
hand_test = []
for i in range(4,5):
    with open(f'test_data/data_{i}.json', encoding='utf-8') as f:
        hand_test.extend(load(f))

hand_test = [Product(**p) for p in hand_test]

In [7]:
from sklearn.svm import OneClassSVM, LinearSVC, SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS


from model import ProductMatchingModel


model = ProductMatchingModel(RidgeClassifier(alpha=0.4),
                             RidgeClassifier(alpha=0.1),
                             RidgeClassifier(alpha=0.1),
                             TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))

model.fit(all_products)

print(accuracy(model.predict(all_products), map(lambda p: p.reference_id, all_products)),
accuracy(model.predict(products_test), map(lambda p: p.reference_id, products_test)),
accuracy(model.predict(hand_test), map(lambda p: p.reference_id, hand_test)),)

0.7463729177861365 0.9474820143884892 0.16666666666666666


In [8]:
model.dump('model.pkl')

In [10]:
model = model.load('model.pkl')

In [14]:


model = ProductMatchingModel(RidgeClassifier(alpha=0.4),
                             RidgeClassifier(alpha=0.1),
                             RidgeClassifier(alpha=0.1),
                             TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))

In [9]:
def classify_error(pred, targ):
    if pred == targ:
        return
    if pred is None and targ is not None:
        return 'freerider'
    if pred is not None and targ is None:
        return 'bastard'
    return 'chameleon'


In [10]:
from collections import Counter


results = list(zip(model.predict(hand_test), map(lambda p: p.reference_id, hand_test)))
cnt = Counter([classify_error(a, b) for a,b in results if a != b])
print('acc=', 1 - sum(cnt.values()) / len(results))
cnt

acc= 0.16666666666666663


Counter({'bastard': 5})

In [11]:
for r, product_given in zip(results, hand_test):
        predicted, target = r
    #if predicted != target:
        print('\n\n<=====================>\n')

        print(product_given)

        print('\n matched with \n')

        print(*filter(lambda p: p.product_id == predicted, references))



<=====================>

Referrer of 2b0ab54004b858af Роутер TP-LINK TL-WR842N (3605e12b229373d6): 
 ['Количество  LAN-портов 4', 'Подключение  к интернету (WAN) внешний модем, Ethernet RJ-45']

 matched with 

Reference Wi-Fi роутер TP-LINK TL-WR842N (2b0ab54004b858af): 
 ['Скорость портов\t100 Мбит/с', 'Количество LAN-портов 4', 'Функции и особенности поддержка IPv6, режим моста, WDS, UPnP AV-сервер', 'Частотный  диапазон устройств Wi-Fi\t2.4 ГГц', 'Макс.  скорость беспроводного соединения 300 Мбит/с', 'Поддержка  USB-модема', 'Подключение   к интернету (WAN) внешний модем, Ethernet RJ-45', 'Стандарт  Wi-Fi 802.11 b (Wi-Fi 1), g (Wi-Fi 3), n (Wi-Fi 4), a (Wi-Fi 2)']


<=====================>

Reference Wi-Fi роутер TP-LINK TL-WR842N (2b0ab54004b858af): 
 ['Скорость портов\t100 Мбит/с', 'Количество LAN-портов 4', 'Функции и особенности поддержка IPv6, режим моста, WDS, UPnP AV-сервер', 'Частотный  диапазон устройств Wi-Fi\t2.4 ГГц', 'Макс.  скорость беспроводного соединения 300 Мбит

In [12]:
0/0

ZeroDivisionError: division by zero

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import clone

from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import STOP_WORDS as EN_STOP_WORDS
from spacy.lang.ru import STOP_WORDS as RU_STOP_WORDS

import pandas as pd
import plotly.express as px

from model import ProductMatchingModel

use_models = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
]

models = list(zip(map(lambda m: m.__class__.__name__, use_models), use_models)) # now its [(name, model), ...]

model_tests_df = []
for name, model in models:
    master_model = ProductMatchingModel(RidgeClassifier(alpha=1.9), clone(model), clone(model),
                                        vectorizer=TfidfVectorizer(stop_words=RU_STOP_WORDS or EN_STOP_WORDS))
    master_model.fit(all_products)
    train_acc, test_acc, hand_acc = accuracy(master_model.predict(all_products), map(lambda p: p.reference_id, all_products)),\
                          accuracy(master_model.predict(products_test), map(lambda p: p.reference_id, products_test)),\
                          accuracy(master_model.predict(hand_test), map(lambda p: p.reference_id, hand_test)),

    print(name, f"train: {train_acc}, test: {test_acc}, hand: {hand_acc}")
    model_tests_df.append([name, train_acc, test_acc, hand_acc])

model_tests_df = pd.DataFrame(model_tests_df, columns=['name', 'train_acc', 'test_acc', 'hand_acc'])

bar = px.bar(model_tests_df, x='name', y=['train_acc', 'test_acc', 'hand_acc'], barmode='group', log_y=False)
bar.show()